In [1]:
import pandas as pd

In [2]:
from urllib.request import urlopen

In [3]:
from bs4 import BeautifulSoup

In [4]:
last_year = 2023

In [5]:
first_year = 1956

In [6]:
nba_mvp_url = "https://www.basketball-reference.com/awards/mvp.html"

In [7]:
per_game_player_stats_url = "https://www.basketball-reference.com/leagues/NBA_{year}_per_game.html" #by year

In [8]:
per_player_advanced_url = "https://www.basketball-reference.com/leagues/NBA_{year}_advanced.html" 

In [9]:
def get_nba_mvp_df():
    nba_mvp_html = urlopen(nba_mvp_url)
    mvp_soup = BeautifulSoup(nba_mvp_html, "html.parser")
    nba_mvp_table = mvp_soup.find('table')
    nba_mvp_table_headers = [th.getText() for th in nba_mvp_table.findAll("tr")[1].findAll('th')]
    nba_mvp_table_rows =  nba_mvp_table.find("tbody").findAll('tr')
    nba_mvp_table_data = [[td.getText() for td in nba_mvp_table_rows[i].findAll('td')] for i in range(len(nba_mvp_table_rows))]
    nba_mvp_table_rows_season = [th.getText() for th in nba_mvp_table.find("tbody").findAll('th')]
    
    ## combining season with other data as season data is nested
    nba_mvp_table_data_total =[]
    for i in range(len(nba_mvp_table_rows_season)):
        nba_mvp_table_data_total.append([])
        nba_mvp_table_data_total[i].append(nba_mvp_table_rows_season[i])
        nba_mvp_table_data_total[i].extend(nba_mvp_table_data[i])
    
    nba_mvp_df = pd.DataFrame(nba_mvp_table_data_total, columns=nba_mvp_table_headers)
    
    return nba_mvp_df
    

In [10]:
#print(nba_mvp_table_data_total)
#print(nba_mvp_table_headers)

nba_mvp_df = get_nba_mvp_df()

In [11]:
nba_mvp_df.head(10)  ## mvp data frame head 

,Season,Lg,Player,Voting,Age,Tm,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48
0,2022-23,NBA,Joel Embiid,(V),28,PHI,66,34.6,33.1,10.2,4.2,1.0,1.7,.548,.330,.857,12.3,.259
1,2021-22,NBA,Nikola Jokić,(V),26,DEN,74,33.5,27.1,13.8,7.9,1.5,0.9,.583,.337,.810,15.2,.296
2,2020-21,NBA,Nikola Jokić,(V),25,DEN,72,34.6,26.4,10.8,8.3,1.3,0.7,.566,.388,.868,15.6,.301
3,2019-20,NBA,Giannis Antetokounmpo,(V),25,MIL,63,30.4,29.5,13.6,5.6,1.0,1.0,.553,.304,.633,11.1,.279
4,2018-19,NBA,Giannis Antetokounmpo,(V),24,MIL,72,32.8,27.7,12.5,5.9,1.3,1.5,.578,.256,.729,14.4,.292
5,2017-18,NBA,James Harden,(V),28,HOU,72,35.4,30.4,5.4,8.8,1.8,0.7,.449,.367,.858,15.4,.289
6,2016-17,NBA,Russell Westbrook,(V),28,OKC,81,34.6,31.6,10.7,10.4,1.6,0.4,.425,.343,.845,13.1,.224
7,2015-16,NBA,Stephen Curry,(V),27,GSW,79,34.2,30.1,5.4,6.7,2.1,0.2,.504,.454,.908,17.9,.318
8,2014-15,NBA,Stephen Curry,(V),26,GSW,80,32.7,23.8,4.3,7.7,2.0,0.2,.487,.443,.914,15.7,.288
9,2013-14,NBA,Kevin Durant,(V),25,OKC,81,38.5,32.0,7.4,5.5,1.3,0.7,.503,.391,.873,19.2,.295


In [12]:
## get data for everyplayer for season this will be used to find averages for statistics

In [13]:
print(per_game_player_stats_url.format(year=2023))

https://www.basketball-reference.com/leagues/NBA_2023_per_game.html


In [14]:
def get_player_year_data_headers(url,year):
    player_data_html = urlopen(url.format(year=year))
    player_soup = BeautifulSoup(player_data_html, "html.parser")
    player_data_table = player_soup.find('table')
    player_table_headers = [th.getText() for th in player_data_table.find('thead').findAll('th')]
    player_table_headers.append('season')
    return player_table_headers

In [16]:
print(get_player_year_data_headers(per_game_player_stats_url,"2023"))

['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'season']


In [28]:
def get_player_year_data_data(url,year):
    player_data_html = urlopen(url.format(year=year))
    player_soup = BeautifulSoup(player_data_html, "html.parser")
    player_data_table = player_soup.find('table')
    player_data_table_rows = player_data_table.find('tbody').findAll('tr')
    #print('player_data_table_rows',player_data_table_rows[0])
    player_data_full = [[td.getText() for td in player_data_table_rows[i].findAll('td')] for i in range(len(player_data_table_rows))]
    
    for i in range(len(player_data_full)):
        player_data_full[i].append(year)
    return player_data_full

In [32]:
print(get_player_year_data_data(per_game_player_stats_url,2023)[0])

HTTPError: HTTP Error 429: Too Many Requests

In [30]:
def get_player_year_data(url):
    headers = get_player_year_data_headers(url,"2023")
    player_data = []
    for year in range(first_year, last_year+1):
        player_data.append(get_player_year_data_data(url,year))
        
    player_df = pd.DataFrame(player_data, columns=headers)
    return player_df

In [31]:
player_per_game_df = get_player_year_data(per_game_player_stats_url)

HTTPError: HTTP Error 429: Too Many Requests